In [1]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

tf.set_random_seed(777)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import requests
url = 'https://biendata.com/competition/airquality/ld/2018-03-01-0/2018-05-31-23/2k0d1d8'
respones= requests.get(url)
with open ("ld_airquality.csv",'w') as f:
    f.write(respones.text)
    
import pandas as pd
l_air = pd.read_csv("ld_airquality.csv")

In [3]:
l_air = l_air[l_air['time'] < '2018-05-18 00:00:00'] # 19일,20일 예측이면 2018-05-18 00:00:00 까지 데이터 받아서 진행

In [4]:
l_air.head(3)

,id,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
0,2941506,CD1,2018-03-31 07:00:00,9.9,14.3,70.4,NaN,NaN,NaN
1,2941507,BL0,2018-03-31 07:00:00,8.5,8.8,44.8,NaN,NaN,NaN
2,2941508,GR4,2018-03-31 07:00:00,4.8,8.9,16.5,NaN,NaN,NaN


In [5]:
l_air.tail(3)

,id,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
21198,3002387,ST5,2018-05-16 23:00:00,6.0,20.0,7.1,NaN,NaN,NaN
21199,3002388,RB7,2018-05-16 23:00:00,11.0,21.2,14.5,NaN,NaN,NaN
21200,3002389,TD5,2018-05-16 23:00:00,7.3,NaN,NaN,NaN,NaN,NaN


In [6]:
l_air.columns = ['id', 'test_id', 'time', 'PM2.5', 'PM10','NO2', 'CO', 'O3','SO2']

In [7]:
sample = pd.read_csv("sample_submission.csv")

def id(i):
    return i.split('#')[0]

test_station = pd.DataFrame(list(map(id, sample['test_id']))).drop_duplicates()
test_station.columns = ['id']
test_station = list(test_station['id'])

bj_test_station = test_station[:35]
ld_test_station = test_station[35:]

In [8]:
def function1(i, y):
    l_air[l_air["test_id"] == i][y].fillna(np.mean(l_air[l_air["test_id"] == i][y]))
    dataset = l_air[l_air["test_id"] == i][y]
    return dataset

In [9]:
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_cell_hidden_dim, 
                                        forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
    if keep_prob < 1.0:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell

In [10]:
def smape(actual, predicted):
    dividend= np.abs(np.array(actual) - np.array(predicted))
    denominator = np.array(actual) + np.array(predicted)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

In [11]:
# 하이퍼파라미터
rnn_cell_hidden_dim = 32   # 각 셀의 (hidden)출력 크기
forget_bias = 1.0          # 망각편향(기본값 1.0)
num_stacked_layers = 1     # stacked LSTM layers 개수
keep_prob = 1.0            # dropout할 때 keep할 비율
epoch_num = 1000           # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.01      # 학습률`

In [12]:
tf.reset_default_graph()

In [16]:
submission = []
y = 'PM10'
for idx,i in enumerate(ld_test_station):
    mean = np.mean(pd.DataFrame(function1(i,y)))
    aq1 = list(pd.DataFrame(function1(i,y)).fillna(round(mean,1))[y])
    
    data=[]
    for i in range(len(aq1)-48):
        aq = aq1[i:i+48]
        data.append(aq)
    data_X=[]
    data_Y=[]
    seq_length = 7
    for i in range(len(data)-seq_length):
        data_X.append(data[i:i+seq_length]) # i+seq_length번째 날 예측
        data_Y.append(data[i+seq_length])
        
     
    train_size = int(len(data_Y) * 0.8)
    # 데이터를 잘라 학습용 데이터 생성
    trainX = data_X[:train_size]
    trainY = data_Y[:train_size]

    # 데이터를 잘라 테스트용 데이터 생성
    testX = data_X[train_size:]
    testY = data_Y[train_size:]
    
    trainX = np.reshape(trainX, [len(trainX),seq_length,48])
    testX = np.reshape(testX, [len(testX),seq_length,48])
    trainY = np.reshape(trainY,[len(trainY),48])
    testY = np.reshape(testY,[len(testY),48])

    # 모델
    X = tf.placeholder(tf.float32, [None, seq_length,48])
    Y = tf.placeholder(tf.float32, [None, 48])

    targets = tf.placeholder(tf.float32, [None, 48])
    predictions = tf.placeholder(tf.float32, [None, 48])

    # num_stacked_layers개의 층으로 쌓인 Stacked RNNs 생성
    stackedRNNs = [lstm_cell() for _ in range(num_stacked_layers)]
    multi_cells = tf.contrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()

    # RNN Cell(여기서는 LSTM셀임)들을 연결
    hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
    print("hypothesis: ", hypothesis)

    hypothesis = tf.contrib.layers.fully_connected(hypothesis[:, -1], 48, activation_fn=tf.nn.relu)
    
    # 손실함수로 평균제곱오차를 사용한다
    loss = tf.reduce_sum(tf.square(hypothesis - Y))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    #rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets, predictions)))

    temp = tf.divide(tf.abs(targets-predictions), targets+predictions)
    smape = 2*tf.reduce_mean(temp)
    
    train_error_summary = []
    test_error_summary = []  # 테스트용 데이터의 오류를 중간 중간 기록한다
    test_predict = ''        # 테스트용데이터로 예측한 결과
 
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # 학습한다
    print('학습을 시작합니다...')

    for epoch in range(epoch_num):
        _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
        if ((epoch+1) % 100 == 0) or (epoch == epoch_num-1): # 100번째마다 또는 마지막 epoch인 경우

            # 학습용데이터로 rmse오차를 구한다
            train_predict = sess.run(hypothesis, feed_dict={X: trainX})
            train_error = sess.run(smape, feed_dict={targets: trainY, predictions: train_predict})
            train_error_summary.append(train_error)
 
            # 테스트용데이터로 rmse오차를 구한다
            test_predict = sess.run(hypothesis, feed_dict={X: testX})
            test_error = sess.run(smape, feed_dict={targets: testY, predictions: test_predict})
            test_error_summary.append(test_error)
        
            print("epoch: {}, train_error: {}, test_error: {}".format(epoch+1,train_error,test_error))
    recent_data = np.reshape(data_X[len(data_X)-seq_length:][0], [-1,seq_length,48])
    test_predict = sess.run(hypothesis, feed_dict={X: recent_data})
    #print("test_predict", test_predict)
    
    submission.append(test_predict[0])
    
    tf.reset_default_graph()

hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 0.5292067527770996, test_error: 0.40741291642189026
epoch: 200, train_error: 0.5406624674797058, test_error: 0.4478989243507385
epoch: 300, train_error: 0.3493761420249939, test_error: 0.30182087421417236
epoch: 400, train_error: 0.3213634788990021, test_error: 0.2906230092048645
epoch: 500, train_error: 0.31870773434638977, test_error: 0.28620675206184387
epoch: 600, train_error: 0.2958075702190399, test_error: 0.2781633734703064
epoch: 700, train_error: 0.2883117198944092, test_error: 0.27235350012779236
epoch: 800, train_error: 0.2791312336921692, test_error: 0.2724643349647522
epoch: 900, train_error: 0.270709365606308, test_error: 0.26060864329338074
epoch: 1000, train_error: 0.26110604405403137, test_error: 0.2557116746902466
hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 0.542868971824646, test_error: 0.4

epoch: 800, train_error: 0.193874791264534, test_error: 0.29729869961738586
epoch: 900, train_error: 0.18232287466526031, test_error: 0.2858612537384033
epoch: 1000, train_error: 0.17709138989448547, test_error: 0.27857843041419983
hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 0.4402711093425751, test_error: 0.49174997210502625
epoch: 200, train_error: 0.35943639278411865, test_error: 0.3719254434108734
epoch: 300, train_error: 0.36202043294906616, test_error: 0.37529513239860535
epoch: 400, train_error: 0.29572877287864685, test_error: 0.35451558232307434
epoch: 500, train_error: 0.21295811235904694, test_error: 0.2965722978115082
epoch: 600, train_error: 0.20325253903865814, test_error: 0.2903001606464386
epoch: 700, train_error: 0.2135559469461441, test_error: 0.286681592464447
epoch: 800, train_error: 0.2127402424812317, test_error: 0.2950681447982788
epoch: 900, train_error: 0.2026999145746231, test_error: 0.2942259

In [17]:
submission

[array([13.255964 , 12.504937 , 12.783932 , 13.300604 , 13.71947  ,
        13.5770645, 13.719639 , 13.880043 , 13.350439 , 14.468194 ,
        14.642073 , 15.058962 , 15.006409 , 15.711311 , 15.473842 ,
        15.583683 , 16.017313 , 16.3015   , 16.536823 , 17.200684 ,
        17.336838 , 17.40574  , 17.898996 , 18.16253  , 18.319817 ,
        18.599823 , 18.686802 , 18.812885 , 19.105015 , 19.674992 ,
        19.73047  , 20.119059 , 20.23506  , 20.51635  , 20.622744 ,
        20.930672 , 21.164005 , 21.516731 , 21.648891 , 21.929914 ,
        22.144035 , 22.286015 , 22.791292 , 22.96428  , 23.002186 ,
        22.948032 , 22.863796 , 23.300335 ], dtype=float32),
 array([12.920391 , 13.048352 , 12.716551 , 12.5574875, 12.369925 ,
        12.571365 , 12.627952 , 12.580409 , 12.411906 , 12.66296  ,
        13.420055 , 13.316925 , 13.274531 , 13.671121 , 13.313092 ,
        14.16613  , 14.547312 , 14.828199 , 15.014258 , 15.666124 ,
        16.431229 , 16.636547 , 17.426407 , 18.011776 ,

In [15]:
PM25 = np.reshape(submission,-1)

In [18]:
PM10 = np.reshape(submission,-1)

In [19]:
pm25 = pd.DataFrame(PM25)
pm10 = pd.DataFrame(PM10)

In [20]:
pm25.to_csv("ld_25_0517.csv")
pm10.to_csv("ld_10_0517.csv")

In [59]:
aq1

[19.7,
 24.5,
 15.6,
 16.3,
 20.2,
 16.1,
 18.8,
 21.4,
 17.4,
 22.6,
 17.5,
 17.8,
 18.7,
 22.0,
 19.7,
 18.4,
 19.0,
 22.8,
 23.8,
 23.0,
 24.2,
 20.3,
 26.7,
 26.3,
 26.2,
 22.6,
 31.4,
 22.5,
 24.6,
 21.0,
 30.2,
 12.6,
 13.5,
 44.3,
 12.4,
 13.4,
 11.2,
 8.7,
 12.1,
 16.7,
 14.0,
 19.6,
 17.4,
 16.4,
 16.6,
 15.3,
 16.6,
 17.2,
 16.4,
 15.7,
 16.0,
 19.2,
 17.7,
 17.1,
 13.9,
 14.7,
 10.4,
 11.4,
 13.1,
 13.6,
 11.2,
 14.8,
 18.4,
 21.0,
 20.0,
 19.3,
 19.5,
 23.8,
 22.1,
 29.0,
 27.8,
 30.3,
 31.0,
 25.2,
 30.3,
 28.3,
 22.6,
 22.3,
 20.6,
 15.5,
 17.6,
 17.2,
 16.6,
 15.0,
 16.2,
 15.6,
 13.7,
 19.6,
 22.4,
 32.9,
 32.0,
 48.7,
 29.5,
 23.9,
 24.6,
 25.4,
 28.3,
 20.9,
 19.9,
 19.6,
 14.5,
 11.2,
 11.3,
 13.3,
 10.5,
 8.7,
 8.3,
 6.9,
 8.5,
 15.8,
 24.2,
 23.7,
 22.4,
 27.6,
 28.1,
 21.4,
 20.0,
 23.4,
 18.6,
 25.9,
 21.1,
 19.9,
 16.1,
 16.3,
 18.1,
 18.6,
 13.3,
 16.3,
 11.6,
 11.7,
 16.4,
 16.1,
 17.0,
 23.0,
 25.9,
 27.2,
 30.2,
 27.9,
 26.9,
 17.0,
 21.0,
 26.9,
 27.1,
 30.